<a href="https://colab.research.google.com/github/HyeyoungKoh/Data-driven-steel-design/blob/main/ML/XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls
# %cd gdrive/My Drive/data-driven HSS

# ! git clone https://github.com/HyeyoungKoh/Data-driven-steel-design


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
drive  gdrive  sample_data


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import r2_score

Create a dataframe

In [2]:
data_url = 'https://raw.githubusercontent.com/HyeyoungKoh/Data-driven-steel-design/main/dataset.csv?token=GHSAT0AAAAAAB44COZWJPDENINJCHSGIKWSY5OFSPA'
data = pd.read_csv(data_url, header=0)

data = data[data!= -1] # remove Nu=-1 (publish data without -1)
data = data.dropna()
data_x = data[['Forming','$H$', '$B$', '$r_o$', '$t$', '$L_{c}$', '$F_y$']] #primary parameters
# data_x = data[['Forming','$A_{e}$', '$I$', '$L_{c}/r$','$F_y$']] #secondary parameters

data['Forming'].replace(['Cold-formed', 'Hot-rolled'],[0, 1], inplace=True) #label


HTTPError: ignored

Split the data into train and test sets

In [ ]:
y = pd.DataFrame(data,columns=['$N_u (kN)$']) #response variable, Nu
X= pd.DataFrame(data,columns=data_x.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123) # can change test_size



Train the XGBoost model

In [ ]:
eval_set = [(X_test, y_test)]
xgb=  XGBRegressor(colsample_bytree= 0.6, gamma= 0.05, learning_rate= 0.2, max_depth= 6, min_child_weight= 5,verbosity = 0) # train with the primary parameters
# xgb=  XGBRegressor(colsample_bytree= 0.6, gamma= 0.0, learning_rate= 0.15, max_depth= 6, min_child_weight= 3,verbosity = 0) # train with the secondary parameters

xgb.fit(X_train, y_train,early_stopping_rounds=1,eval_set=eval_set, verbose=False)

y_pred_xgb_train = xgb.predict(X_train) #prediction of the training set
y_pred_xgb=xgb.predict(X_test) #prediction of the test set

r2_train = r2_score(y_train, y_pred_xgb_train) # performance of the training set
r2_test = r2_score(y_test, y_pred_xgb) # performance of the test set

print('r2 score for training = {:8.4f}'.format(r2_train))
print('r2 score for testing = {:8.4f}'.format(r2_test))

r2 score for training =   0.9957
r2 score for testing =   0.9885
